In [225]:
import os
# print("Installing tesseract on machine")

# os.system('apt-get install tesseract-ocr -y')

# print("tesseract should be installed")

# os.system('pip install gradio --upgrade')
os.system('pip install git+https://github.com/huggingface/transformers.git --upgrade')
# os.system('pip install pyyaml==5.1')
# # workaround: install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9 (issue: https://github.com/facebookresearch/detectron2/issues/3158)
# os.system('pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html')
# # install detectron2 that matches pytorch 1.8
# # See https://detectron2.readthedocs.io/tutorials/install.html for instructions
# os.system('pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html')
# ## install PyTesseract
# os.system('pip install -q pytesseract')


import time
import numpy as np
from transformers import LayoutLMv2Processor, LayoutLMv2ForTokenClassification,LayoutLMv2FeatureExtractor
from datasets import load_dataset
from PIL import Image, ImageDraw, ImageFont
import uuid

dataset = load_dataset("nielsr/funsd", split="test")

# define id2label, label2color
labels = dataset.features['ner_tags'].feature.names
id2label = {v: k for v, k in enumerate(labels)}
label2color = {'question':'blue', 'answer':'green', 'header':'orange', 'other':'violet'}
l2l = {'question':'key', 'answer':'value', 'header':'title'}
f_labels = {'question':'key', 'answer':'value', 'header':'title', 'other':'others'}


processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")
model = LayoutLMv2ForTokenClassification.from_pretrained("nielsr/layoutlmv2-finetuned-funsd",ignore_mismatched_sizes = True)
feature_extractor = LayoutLMv2FeatureExtractor()


Reusing dataset funsd (/Users/mspraja/.cache/huggingface/datasets/nielsr___funsd/funsd/1.0.0/8b0472b536a2dcb975d59a4fb9d6fea4e6a1abe260b7fed6f75301e168cbe595)


In [300]:
def iob_to_label(label):
    label = label[2:]
    if not label:
      return 'other'
    return label

def unnormalize_box(bbox, width, height):
     return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
     ]

import json
def normalize_box(box, width, height):
    return [
        int(1000 * (box[0] / width)),
        int(1000 * (box[1] / height)),
        int(1000 * (box[2] / width)),
        int(1000 * (box[3] / height)),
    ]

def parsing(true_predictions,token_boxes,bbdict):  
  cluster_master=[]
  cluster=[]
  picked=[]
  prev=''

  for prediction,tbox in zip(true_predictions,token_boxes):
    x,y=tbox[:2]
    if tbox not in picked:
      picked.append(tbox)
    else:
      continue
    try:

      word=bbdict[str(tbox)]
      gap=False
      if x-prevx > 150 or y-prevy > 20 :
        gap=True

      if prediction in ['B-QUESTION','I-QUESTION']:
        if prev=='value':
          cluster_master.append((cluster,'value',x,y))
          cluster=[]
        elif gap:
          cluster_master.append((cluster,prev,x,y))
          cluster=[]

        cluster.append(word)
        prev='key'
      else:
        if prev=='key' :
          cluster_master.append((cluster,'key',x,y))
          cluster=[]
        elif gap:
          cluster_master.append((cluster,prev,x,y))
          cluster=[]
        cluster.append(word)
        prev='value'
      
      
    except:
      pass
    prevx=x
    prevy=y
    
  cluster_master.append((cluster,prev,x,y))
  # return cluster_master


  key_value=dict()
  for item in cluster_master:
    typ=item[1]
    text=' '.join(item[0])
    x,y=item[2:]
    
    if typ=='value' and prevtyp=='key':
      key_value[prevtext]=text


    prevtext=text
    prevtyp=typ
    prevx=x
    prevy=y

  return key_value,cluster_master



def run(image):
    
  image = image
  width, height = image.size
  features = feature_extractor(image, return_tensors="pt")
  words,boxes=features['words'][0],features['boxes'][0]

  bbdict=dict()
  for word,box in zip(words,boxes):
    bbdict[str(box)]=word

  start=time.time()
  encoding = processor(image, words, boxes=boxes, return_tensors="pt")
  outputs = model(**encoding)
  print(time.time()-start,'Deep learning')


  # get predictions
  predictions = outputs.logits.argmax(-1).squeeze().tolist()
  token_boxes = encoding.bbox.squeeze().tolist()
  # print('token_boxes',token_boxes)

  # only keep non-subword predictions
  true_predictions = [id2label[pred] for idx, pred in enumerate(predictions)]
  print(len(true_predictions))
  true_boxes = [unnormalize_box(box, width, height) for idx, box in enumerate(token_boxes)]
  print(len(true_boxes))

    # draw predictions over the image
  draw = ImageDraw.Draw(image)
  font = ImageFont.load_default()
  for prediction, box in zip(true_predictions, true_boxes):
      predicted_label = iob_to_label(prediction).lower()
      draw.rectangle(box, outline=label2color[predicted_label])
      draw.text((box[0]+10, box[1]-10), f_labels[predicted_label], fill=label2color[predicted_label], font=font)
  output,cluster_master=parsing(true_predictions,token_boxes,bbdict)
  filename = str(uuid.uuid4())+'.jpg'
  image.save(filename)
  return output

In [301]:
##input images here
im = Image.open('/Users/mspraja/FUNSD-Entity-Linking/dataset/testing_data/images/82250337_0338.png').convert("RGB")


In [302]:
im


In [303]:
run(im)

1.586597204208374 Deep learning
345
345


{'To:': 'SamZolot',
 'MANUFACTURER:': '__BaW',
 'FROM:': 'D.J. Lando',
 'BRAND:': 'Kool Waterfall',
 'DATE:': '“2.08e97',
 'TYPE OF PACKINGS:': 'Al Packing',
 'REPORTING PERIODS:': 'oot Now X ee. Jan.',
 '‘TEST MARKET GEOGRAPHY:': 'oivisonse21 andsar  (Cwsiscomsim)',
 'PRICE POINT: FULLS _PIVS': '(indicat Distrbutor’s Cost Per Caron)',
 'SALES FORCE INVOLVEME?': 'They have ctew-worked distribution, and itis reported that they may crew-workit again. Sales force has been busy',
 'DISTRIBUTORS - ACCEPTANCEMNTRO TERMS/INTRO DEALS/INVOLVEMENT:': '‘Allaccounts have the new packaging. Itwas not a problem obtaining new distribution. All accounts appear',
 'CHAINS - ACCEPTANCE/MERCHANDISING:': '‘This has not been a problem. New packaging is just following up on the old ‘packaging’.',
 'INDEPENDENTS - ACCEPTANCE/MERCHANDISING:': 'Very well received. The old packs are boing consolidated and promoted in select retail cations at 40¢',
 'ADVERTISING - EFFECTIVENESS OF P.0.S.:': '“The theme "B* Koo! 